In [20]:
import requests
import json

def test_local_server():
    """Test local server trước khi deploy"""
    base_url = "http://localhost:8080"  # Local server
    
    # Test health
    print("=== TESTING LOCAL SERVER ===")
    try:
        health_response = requests.get(f"{base_url}/health", timeout=5)
        print(f"Health: {health_response.status_code} - {health_response.json()}")
    except Exception as e:
        print(f"❌ Local server not running: {e}")
        print("Chạy: npm run dev")
        return
    
    # Test process document
    test_data = {
        "recordId": "recv6dq3ft",
        "appToken": "Z2cxb4FTWaYhXTsJdCGlDAYYgYq",
        "tableID": "tblpzAwC8Ol4VWmZ",
        "idFieldName": "ID",
        "loaiVanBanFieldName": "Loại văn bản - con",
        "hoSoDinhKemFieldName": "Tài liệu đính kèm",
        "creatorOpenId": "ou_41ff6114c70aef421d5e9d70953bd208"
    }
    
    try:
        response = requests.post(f"{base_url}/process-document", 
            json=test_data, 
            timeout=30
        )
        
        print(f"\nProcess Document Status: {response.status_code}")
        print(f"Response: {json.dumps(response.json(), indent=2, ensure_ascii=False)}")
        
    except Exception as e:
        print(f"❌ Local test failed: {e}")

if __name__ == "__main__":
    test_local_server()


=== TESTING LOCAL SERVER ===
Health: 200 - {'status': 'OK', 'timestamp': '2025-06-26T09:34:37.826Z'}

Process Document Status: 200
Response: {
  "success": true,
  "instanceCode": "84385131-2981-406E-ADAB-8766E0D9C624",
  "message": "Đã tạo thành công đơn phê duyệt văn bản với 1 tài liệu đính kèm",
  "documentInfo": {
    "loaiVanBan": "Công văn nội bộ"
  },
  "uploadedCodes": [
    "6FA7AAC5-699A-426D-B2DB-C3538B22862A"
  ],
  "documentCount": 1
}
